# Setting up the Fast Withdrawal contract:
- This notebook allows deploying contracts for fast withdrawal interactions and running a basic scenario by creating a withdrawal on the Etherlink side and making fast payouts on the Tezos side.

In [1]:
from docs.scenarios.setup import *

web3, etherlink_account, tezos_account = setup()

Setup:
- Tezos account: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`, balance: `228.452887 ꜩ`
- Etherlink account: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`, balance: `610.100075527 ꜩ`


## Deploy FastWithdrawal:

In [2]:
FAST_WITHDRAWAL_CONTRACT = 'KT19MJZvZLBagbtBapBcfWqkECu9B93RAGDp'

In [3]:
from scripts.tezos import deploy_fast_withdrawal
from scripts.helpers.contracts.fast_withdrawal import FastWithdrawal

if FAST_WITHDRAWAL_CONTRACT:
    fast_withdrawal = FastWithdrawal.from_address(tezos_account, FAST_WITHDRAWAL_CONTRACT)
    click.echo('Loaded Fast Withdrawal contract, address: ' + wrap(accent(fast_withdrawal.address)))
else:
    fast_withdrawal = deploy_fast_withdrawal.callback(
        xtz_ticketer_address=XTZ_TICKETER_ADDRESS,
        smart_rollup_address=SMART_ROLLUP_ADDRESS,
        tezos_private_key=TEZOS_PRIVATE_KEY,
        tezos_rpc_url=TEZOS_RPC_URL,
        skip_confirm=True,
        silent=False,
    )

Deploying Fast Withdrawal contract:
  - Deployer: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Tezos RPC node: `https://rpc.tzkt.io/ghostnet/`
  - Params:
      * Native XTZ Ticketer address: `KT1Bp9YUvUBJgXxf5UrYTM2CGRUPixURqx4m`
      * Smart Rollup address: `sr18wx6ezkeRjt1SZSeZ2UQzQN3Uc3YLMLqg`
Successfully deployed Fast Withdrawal, address: `KT19MJZvZLBagbtBapBcfWqkECu9B93RAGDp`


## Making Fast Withdrawal on Etherlink side:

In [4]:
from scripts.etherlink import xtz_fast_withdraw

tx_hash = xtz_fast_withdraw.callback(
    target=get_address(tezos_account),
    fast_withdrawal_contract=fast_withdrawal.address,
    amount=717 * 10**12,
    discounted_amount=700,
    withdraw_precompile=XTZ_WITHDRAWAL_PRECOMPILE,
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL,
)
tx_hash

Making Fast Withdrawal, XTZ:
  - Sender: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://node.ghostnet.etherlink.com`
  - Withdrawal params:
      * Target: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
      * Fast Withdrawal contract: `KT19MJZvZLBagbtBapBcfWqkECu9B93RAGDp`
      * Payload bytes: `0500bc0a`
      * Amount (mutez): `717`
      * Discounted amt (mutez): `700`
      * Fee (mutez): `17`
Successfully initiated XTZ Fast Withdrawal, tx hash: `0xb839d1a134f589c6ec65e500a145cca0c491f27707eba86805beb3e4bb6c2cb7`


'0xb839d1a134f589c6ec65e500a145cca0c491f27707eba86805beb3e4bb6c2cb7'

### Searching for outbox message:
- TODO: Move this code to a separate script

In [5]:
from scripts.helpers.rollup_node import get_messages
import time
from pytezos import MichelsonType
from pytezos import michelson_to_micheline


def make_event_logs_link(tx_hash):
    return f'https://testnet.explorer.etherlink.com/tx/{tx_hash}?tab=logs'

def make_outbox_message_link(level):
    return f'https://ghostnet-smart.tzkt.io/global/block/{level}/outbox/{level}/messages'

def make_tzkt_ghostnet_link(tx_hash):
    return f'https://ghostnet.tzkt.io/{tx_hash}'

FAST_WITHDRAWAL_OUTBOX_INTERFACE = '''
pair %default (nat %withdrawal_id)
    (pair (ticket %ticket (pair nat (option bytes)))
         (pair (timestamp %timestamp)
               (pair (address %base_withdrawer)
                     (pair (bytes %payload) (bytes %l2_caller)))))
'''

def decode_outbox_message(message):
    micheline_expression = michelson_to_micheline(FAST_WITHDRAWAL_OUTBOX_INTERFACE)
    michelson_type = MichelsonType.match(micheline_expression)
    assert len(message['transactions']) == 1
    parameters_micheline = message['transactions'][0]['parameters']
    return michelson_type.from_micheline_value(parameters_micheline).to_python_object()

def scan_outbox_until_message_found(
    etherlink_rollup_node_url: str,
    last_level: int,
    max_levels: int = 100,
    sleep_time: int = 0.3
):
    click.echo('Scanning outbox: ', nl=False)
    for level in range(last_level, last_level-max_levels, -1):
        click.echo(wrap(accent(level)), nl=False)
        messages = get_messages(ETHERLINK_ROLLUP_NODE_URL, level)
        if len(messages) > 0:
            click.echo()
            click.echo('Found outbox message at level: ' + wrap(accent(level)))
            return messages
        click.echo(', ', nl=False)
        time.sleep(sleep_time)

def make_withdrawal_info(tx_hash, outbox_message):
    click.echo('Fast Withdrawal info:')
    click.echo('- Event logs: ' + make_event_logs_link(tx_hash))
    click.echo('- Outbox message: ' + make_outbox_message_link(tx_hash))
    click.echo('- Withdrawal ID: ' + wrap(accent(outbox_message['withdrawal_id'])))
    click.echo('- Timestamp: ' + wrap(accent(outbox_message['timestamp'])))

In [6]:
last_tezos_level = tezos_account.shell.head()['header']['level']
messages = scan_outbox_until_message_found(ETHERLINK_ROLLUP_NODE_URL, last_tezos_level)
outbox_message = decode_outbox_message(messages[0]['message'])
make_withdrawal_info(tx_hash, outbox_message)

Scanning outbox: `11689476`, `11689475`, `11689474`, `11689473`, `11689472`, `11689471`, `11689470`, `11689469`, `11689468`, `11689467`
Found outbox message at level: `11689467`
Fast Withdrawal info:
- Event logs: https://testnet.explorer.etherlink.com/tx/0xb839d1a134f589c6ec65e500a145cca0c491f27707eba86805beb3e4bb6c2cb7?tab=logs
- Outbox message: https://ghostnet-smart.tzkt.io/global/block/0xb839d1a134f589c6ec65e500a145cca0c491f27707eba86805beb3e4bb6c2cb7/outbox/0xb839d1a134f589c6ec65e500a145cca0c491f27707eba86805beb3e4bb6c2cb7/messages
- Withdrawal ID: `1955`
- Timestamp: `1744097875`


## Purchasing withdrawal on Tezos side:

In [7]:
from scripts.helpers.contracts.fast_withdrawal import Withdrawal
from scripts.helpers.utility import pack
from scripts.helpers.ticket_content import TicketContent

discounted_amount = 700

withdrawal = Withdrawal(
    withdrawal_id=outbox_message['withdrawal_id'],
    full_amount=717,
    ticketer=XTZ_TICKETER_ADDRESS,
    content=TicketContent(
        token_id=0,
        token_info=None,
    ),
    timestamp=outbox_message['timestamp'],
    base_withdrawer=get_address(tezos_account),
    payload=pack(discounted_amount, 'nat'),
    l2_caller=bytes.fromhex(etherlink_account.address.split('0x')[1])
)

opg = fast_withdrawal.payout_withdrawal(
    withdrawal=withdrawal,
    service_provider=get_address(tezos_account),
    xtz_amount=discounted_amount,
).send()

click.echo('- Transaction: ' + make_tzkt_ghostnet_link(opg.hash()))

- Transaction: https://ghostnet.tzkt.io/op6q7NyK1mi6VoVZy3228GWvVziGPQbhf1uKHCXQUgzUGeZEz2p


## Making Additional Withdrawals:
### One with an incorrect payload that would be impossible to execute

In [8]:
from scripts.etherlink.xtz_fast_withdraw import (
    load_withdraw_precompile,
    make_fast_withdrawal,
)

precompile_contract = load_withdraw_precompile(
    XTZ_WITHDRAWAL_PRECOMPILE,
    web3
)
receipt = make_fast_withdrawal(
    etherlink_account=etherlink_account,
    web3=web3,
    precompile_contract=precompile_contract,
    target=get_address(tezos_account),
    fast_withdrawals_contract=fast_withdrawal.address,
    payload=bytes.fromhex('ff4f'),
    wei_amount=133 * 10**12,
)

click.echo(
    'Successfully initiated wrong XTZ Fast Withdrawal, tx hash: '
    + wrap(accent(receipt.transactionHash.hex()))
)

Successfully initiated wrong XTZ Fast Withdrawal, tx hash: `0xd48eea19ee4176da9743d805b02a84d05b0af0cfa71a60791f9c3c258cdac07a`


### Three more with different base withdrawers and amounts

In [19]:
from random import randint

for num in range(1, 4):
    click.echo('--- W I T H D R A W A L   :   ' + accent(num))

    full_amount = randint(100, 1000)
    full_amount_wei = full_amount * 10**12
    discounted_amount = int(full_amount * 0.95)
    random_target = tezos_account.key.generate(export=False).public_key_hash()

    tx_hash = xtz_fast_withdraw.callback(
        target=random_target,
        fast_withdrawal_contract=fast_withdrawal.address,
        amount=full_amount_wei,
        discounted_amount=discounted_amount,
        withdraw_precompile=XTZ_WITHDRAWAL_PRECOMPILE,
        etherlink_private_key=ETHERLINK_PRIVATE_KEY,
        etherlink_rpc_url=ETHERLINK_RPC_URL,
    )

    time.sleep(20)
    last_tezos_level = tezos_account.shell.head()['header']['level']
    messages = scan_outbox_until_message_found(ETHERLINK_ROLLUP_NODE_URL, last_tezos_level)
    outbox_message = decode_outbox_message(messages[0]['message'])
    make_withdrawal_info(tx_hash, outbox_message)

    # TODO: `Withdrawal.from_outbox`?
    withdrawal = Withdrawal(
        withdrawal_id=outbox_message['withdrawal_id'],
        full_amount=full_amount,
        ticketer=XTZ_TICKETER_ADDRESS,
        content=TicketContent(
            token_id=0,
            token_info=None,
        ),
        timestamp=outbox_message['timestamp'],
        base_withdrawer=random_target,
        payload=pack(discounted_amount, 'nat'),
        l2_caller=bytes.fromhex(etherlink_account.address.split('0x')[1])
    )

    opg = fast_withdrawal.payout_withdrawal(
        withdrawal=withdrawal,
        service_provider=get_address(tezos_account),
        xtz_amount=discounted_amount,
    ).send()

    click.echo('- Payout transaction: ' + make_tzkt_ghostnet_link(opg.hash()))

--- W I T H D R A W A L   :   1
Making Fast Withdrawal, XTZ:
  - Sender: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://node.ghostnet.etherlink.com`
  - Withdrawal params:
      * Target: `tz1R4spYiySApAai2jQduadYrQSvVHSNxZdm`
      * Fast Withdrawal contract: `KT19MJZvZLBagbtBapBcfWqkECu9B93RAGDp`
      * Payload bytes: `0500a502`
      * Amount (mutez): `174`
      * Discounted amt (mutez): `165`
      * Fee (mutez): `9`
Successfully initiated XTZ Fast Withdrawal, tx hash: `0xb690d8fcc490daf3caef069f7c3e76562fead589d0cf23fc86f1a8a995646db3`
Scanning outbox: `11689593`, `11689592`, `11689591`, `11689590`
Found outbox message at level: `11689590`
Fast Withdrawal info:
- Event logs: https://testnet.explorer.etherlink.com/tx/0xb690d8fcc490daf3caef069f7c3e76562fead589d0cf23fc86f1a8a995646db3?tab=logs
- Outbox message: https://ghostnet-smart.tzkt.io/global/block/0xb690d8fcc490daf3caef069f7c3e76562fead589d0cf23fc86f1a8a995646db3/outbox/0xb690d8fcc490daf3caef06